In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
def solve(payoffs: dict, max_tasks: dict, initial_solution: dict, num_teams = int, num_sites = int):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")

        
        
        N =  [i for i in range(1, num_teams + 1)]
        J = [j for j in range(1, num_sites + 1)]
        
        x = {}
        a = payoffs
        
        
        for i in N:
            for j in J:
                    x[i, j] = m.addVar(vtype=GRB.BINARY, name="x_%s_%s" % (i, j))
        
                    
        objective = gp.quicksum(a[i, j] * x[i, j] for i in N for j in J)
        m.setObjective(objective, GRB.MAXIMIZE)
        

        
        for j in J:
            m.addConstr(gp.quicksum(x[i, j] for i in N) <= 1, name=f"c_{j}")
            
        for i in N:
            m.addConstr(gp.quicksum(x[i, j] for j in J) <= max_tasks[i-1], name=f"c_{i}")
            m.addConstr(x[i, j] * a[i,j] >= initial_solution[i, j] * a[i,j], name=f"c_{i}_{j}")    
            
        
        m.optimize()

                
        if m.status == GRB.OPTIMAL:
            m.getAttr('x', x)   
            print('Obj: %g' % m.objVal)
            

        return m.objVal
    
    
# FINDING z*
    
def solve_extra_constraint(payoffs: dict, max_tasks: dict, initial_solution: dict, num_teams: int, num_sites: int, extra_constraint: tuple):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")

        
        
        N =  [i for i in range(1, num_teams + 1)]
        J = [j for j in range(1, num_sites + 1)]
        
        x = {}
        a = payoffs
        
        
        for i in N:
            for j in J:
                    x[i, j] = m.addVar(vtype=GRB.BINARY, name="x_%s_%s" % (i, j))
        
                    
        objective = gp.quicksum(a[i, j] * x[i, j] for i in N for j in J)
        m.setObjective(objective, GRB.MAXIMIZE)
        

        
        for j in J:
            m.addConstr(gp.quicksum(x[i, j] for i in N) <= 1, name=f"c_{j}")
            
        for i in N:
            m.addConstr(gp.quicksum(x[i, j] for j in J) <= max_tasks[i-1], name=f"c_{i}")
            m.addConstr(x[i, j] * a[i,j] >= initial_solution[i, j] * a[i,j], name=f"c_{i}_{j}")    
        
        x_i, x_j = extra_constraint
        
        m.addConstr(x[x_i, x_j] == 1, name=f"c_{x_i}_{x_j}")

        
        m.optimize()

                
        if m.status == GRB.OPTIMAL:
            m.getAttr('x', x)   
            print('Obj: %g' % m.objVal)
            

        return m.objVal

In [3]:
def expected_payoff(all_lambdas, n_teams, n_sites, team_sizes):
    alpha, beta, gamma, delta = 1.451, 0.138, -0.287, 0.057
    #Same Shape as SiteTeamMatrix
    expected_rewards = np.empty((n_teams, n_sites)) * 0.0

    k = 5

    for i in range(n_teams):
        team_size = team_sizes[i]
        for j in range(n_sites):
            # FIX THIS: 
            # Furthermore, STsji denotes the estimated starting time (in hours) of team i at site sj, which Van Rijn et al. (2020) estimate as 7 AM plus the driving time
            expected_rewards[i][j] = all_lambdas[j] * (alpha + beta * (np.sqrt(12 * k) + np.sqrt(12 * (k - 1))) + gamma * np.sqrt(12) + delta * team_size)
    
    return expected_rewards

In [4]:
def inititial_sol(SiteTeam: pd.DataFrame, n_teams: int, n_sites: int) -> dict:
    initial_solution = {}
    shape_params = SiteTeam.shape  
    
    for i in range(1, shape_params[0] + 1):
        for j in range(1, shape_params[1] + 1):
            initial_solution[i, j] = SiteTeam.iloc[i-1, j-1]
    initial_solution1 = {(j, i): initial_solution[i, j] for i in range(1, n_sites + 1) for j in range(1, n_teams + 1)}

    return initial_solution1

In [10]:
obj = {}
    
initial_solutions = {}
for i in range(11, 12):
    print(f"Solving case {i}")
    path = f"../Generated cases/case{i}.xlsx"

    Team = pd.read_excel(path, sheet_name="Team")
    SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
    Site = pd.read_excel(path, sheet_name="Site")
    DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
    DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)
    feasible_locations = (DistanceSiteTeam <= 60).astype(int)
    
    info = pd.read_excel(path, sheet_name="Info")
    n_teams, n_sites = info.values[0]

    print(f"Number of teams: {n_teams}, Number of sites: {n_sites}")
    team_sizes = Team.iloc[:, 1]
    all_lambdas = Site.iloc[:, 4]
    expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites, team_sizes)
    expected_rewards = feasible_locations.T.values * expected_rewards

    initial_solution = inititial_sol(SiteTeam, n_teams, n_sites)
    
    
    shape_params = expected_rewards.shape

    payoff = {(i, j): expected_rewards[i-1 , j-1] for i in range(1, shape_params[0] + 1) for j in range(1, shape_params[1] + 1)}
    max_tasks = [400 for i in range(n_teams)]

    """"
    SOLVING THE OPTIMISATION PROBLEM
    """

    obj[i] = solve(payoff, max_tasks, initial_solution, shape_params[0], shape_params[1])


Solving case 11
Number of teams: 29, Number of sites: 1524
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1582 rows, 44196 columns and 88393 nonzeros
Model fingerprint: 0x04504d3d
Variable types: 0 continuous, 44196 integer (44196 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [6e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 116409.52924
Presolve removed 1582 rows and 44196 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 116560 116410 

Optimal solution found (tolerance 1.00e-04

In [277]:
constraint_sol = {}

path = f"../Generated cases/case{1}.xlsx"

Team = pd.read_excel(path, sheet_name="Team")
SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
Site = pd.read_excel(path, sheet_name="Site")
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)
feasible_locations = (DistanceSiteTeam <= 60).astype(int)

info = pd.read_excel(path, sheet_name="Info")
n_teams, n_sites = info.values[0]

print(f"Number of teams: {n_teams}, Number of sites: {n_sites}")
team_sizes = Team.iloc[:, 1]
all_lambdas = Site.iloc[:, 4]
expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites, team_sizes)
expected_rewards = feasible_locations.T.values * expected_rewards

initial_solution = inititial_sol(SiteTeam, n_teams, n_sites)


shape_params = expected_rewards.shape

payoff = {(i, j): expected_rewards[i-1 , j-1] 
          for i in range(1, shape_params[0] + 1) 
          for j in range(1, shape_params[1] + 1)}
max_tasks = [20 for i in range(n_teams)]

""""
SOLVING THE OPTIMISATION PROBLEM
"""

sol  = solve(payoff, max_tasks, initial_solution, shape_params[0], shape_params[1])


for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        
        #IF INFEASIBLE LOCATION MAXIMISATION DOES NOT WORK
        if feasible_locations.iloc[j - 1, i -1] == 0:
            constraint_sol[i, j] = sol 
        
        else:
            constraint_sol[i, j] = solve_extra_constraint(payoff, max_tasks, initial_solution, shape_params[0], shape_params[1], (i, j))


Number of teams: 19, Number of sites: 944
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x183e6d4c
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 10746.582733
Presolve removed 417 rows and 16621 columns
Presolve time: 0.02s
Presolved: 565 rows, 1315 columns, 2554 nonzeros
Found heuristic solution: objective 16563.719802
Variable types: 0 continuous, 1315 integer (1315 binary)
Found heuristic solution: objective 16771.961698

Root relaxation: objective 1.695328e+04, 698 iterati

KeyboardInterrupt: 

In [23]:
import numpy as np
feasible_locations.iloc[0, 2]

1

In [17]:
feasible_locations

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# FORMULATION WITH INCENTIVE 

In [27]:
def solve_with_incentive(payoffs: dict, max_tasks: dict, initial_solution: dict, num_teams: int, num_sites: int, incentive_scheme):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")

        
        
        N =  [i for i in range(1, num_teams + 1)]
        J = [j for j in range(1, num_sites + 1)]
        
        x = {}
        a = payoffs
        p = incentive_scheme
        
        for i in N:
            for j in J:
                    x[i, j] = m.addVar(vtype=GRB.BINARY, name="x_%s_%s" % (i, j))
        
                    
        objective = gp.quicksum(a[i, j] * x[i, j] * p[i, j] for i in N for j in J)
        m.setObjective(objective, GRB.MAXIMIZE)
        

        
        for j in J:
            m.addConstr(gp.quicksum(x[i, j] for i in N) <= 1, name=f"c_{j}")
            
        for i in N:
            m.addConstr(gp.quicksum(x[i, j] for j in J) <= max_tasks[i-1], name=f"c_{i}")
            m.addConstr(x[i, j] * a[i,j] * p[i, j]>= initial_solution[i, j] * a[i,j] * p[i, j], name=f"c_{i}_{j}")    
            

        
        m.optimize()

                
        if m.status == GRB.OPTIMAL:
            m.getAttr('x', x)   
            print('Obj: %g' % m.objVal)
            

        return m.objVal

# DUAL PROBLEM

In [92]:
def dual(max_tasks, num_sites, num_teams, payoff):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")
        
        J = [j for j in range(1, num_sites + 1)]
        N = [i for i in range(1, num_teams + 1)]
        
        v = {}
        w = {}
        
        
        for i in N:
            v[i] = m.addVar(vtype = GRB.CONTINUOUS)

        for j in J:
            w[j] = m.addVar(vtype = GRB.CONTINUOUS)
            
        m.setObjective(gp.quicksum(max_tasks[i - 1] * v[i] for i in N) + gp.quicksum(w[j] for j in J), GRB.MINIMIZE)
        
        
        for i in N:
            for j in J:
                m.addConstr(v[i] + w[j] >= payoff[i, j], name=f"c_{i}_{j}")
                
                
        for i in N:
            m.addConstr(v[i] >= 0, name=f"c_{i}")
        for j in J:
            m.addConstr(w[j] >= 0, name=f"c_{j}")
        
        
        m.optimize()
        
        
        if m.status == GRB.OPTIMAL:
            print('Obj: %g' % m.objVal)
            v_values = {i: v[i].x for i in N}
            w_values = {j: w[j].x for j in J}


        return m.objVal, v_values, w_values
    

# FIRST INCENTIVE

In [83]:
z_optimal = sol 
z_star = constraint_sol

#ADJUST THIS LATER
c = 1000


incentive = {}

for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        if payoff[i, j] != 0:
            incentive[i, j] = c / (float(payoff[i, j]) * max_tasks[i - 1]) * ((z_optimal - min(constraint_sol.values())) / z_optimal + c * (z_optimal - z_star[i, j]) / z_optimal)
        else: 
            #BECAUSE OF INFEASIBLE LOCATIONS
            incentive[i, j] = 0

        

In [84]:
solve_with_incentive(payoff, max_tasks, initial_solution, n_teams, n_sites, incentive)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x85bff702
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [7e-05, 1e+00]
  Objective range  [7e-05, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 3.3693484
Presolve removed 982 rows and 17936 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 5.91127 3.36935 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.911267761083e+00, best bound 5.911267761083

5.911267761082926

# RETRIEVING r_ij THROUGH DUAL

In [98]:
max_tasks = [50 for i in range(n_teams)]
_, v, w = dual(max_tasks, n_sites, n_teams, payoff)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18899 rows, 963 columns and 36835 nonzeros
Model fingerprint: 0x43f5752e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+02]
Presolve removed 17307 rows and 44 columns
Presolve time: 0.01s
Presolved: 1592 rows, 919 columns, 3184 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 18
 AA' NZ     : 2.457e+03
 Factor NZ  : 5.950e+03 (roughly 1 MB of memory)
 Factor Ops : 2.600e+04 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.01 seconds (0.01 w

In [103]:
R = {(i, j): payoff[i, j] - v[i] - w[j] for i in range(1, n_teams + 1) for j in range(1, n_sites + 1)}

# INCENTIVE 2

In [117]:
c = 10

incentive_2 = {}

for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        if payoff[i, j] != 0:
            incentive_2[i, j] = c/(float(payoff[i, j]) * max_tasks[i - 1]) * (R[i, j] - min(R.values()))
        else:
            incentive_2[i, j] = 0

In [118]:
solve_with_incentive(payoff, max_tasks, initial_solution, n_teams, n_sites, incentive_2)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0xb6533d76
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [7e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 79687.438172
Presolve removed 468 rows and 16787 columns
Presolve time: 0.01s
Presolved: 514 rows, 1149 columns, 2266 nonzeros
Found heuristic solution: objective 80060.346898
Variable types: 0 continuous, 1149 integer (1133 binary)
Found heuristic solution: objective 80998.389674

Root relaxation: objective 8.186270e+04, 90 iterations, 0.00 seconds (0.00 work units)

    No

81862.70146035317

# OPTIMAL SWAPS

In [125]:
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
DistanceTeamTeam = pd.read_excel(path, sheet_name="DistanceTeamTeam")

In [127]:
DistanceTeamTeam.drop("Unnamed: 0", axis=1, inplace=True)

In [141]:
DistanceTeamTeam

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,70.710678,50.000000,212.132034,150.000000,111.803399,100.000000,180.277564,158.113883,223.606798,50.000000,70.710678,50.000000,100.000000,180.277564,180.277564,111.803399,111.803399,206.155281
1,70.710678,0.000000,50.000000,282.842712,206.155281,180.277564,70.710678,250.000000,223.606798,254.950976,111.803399,141.421356,50.000000,158.113883,206.155281,250.000000,150.000000,150.000000,269.258240
2,50.000000,50.000000,0.000000,250.000000,200.000000,158.113883,50.000000,223.606798,206.155281,206.155281,70.710678,111.803399,70.710678,111.803399,158.113883,212.132034,100.000000,158.113883,223.606798
3,212.132034,282.842712,250.000000,0.000000,150.000000,111.803399,291.547595,50.000000,100.000000,254.950976,180.277564,141.421356,250.000000,158.113883,250.000000,50.000000,206.155281,206.155281,111.803399
4,150.000000,206.155281,200.000000,150.000000,0.000000,70.710678,250.000000,100.000000,50.000000,320.156212,158.113883,111.803399,158.113883,180.277564,291.547595,158.113883,223.606798,70.710678,223.606798
5,111.803399,180.277564,158.113883,111.803399,70.710678,0.000000,206.155281,70.710678,50.000000,250.000000,100.000000,50.000000,141.421356,111.803399,223.606798,100.000000,158.113883,100.000000,158.113883
6,100.000000,70.710678,50.000000,291.547595,250.000000,206.155281,0.000000,269.258240,254.950976,200.000000,111.803399,158.113883,111.803399,141.421356,150.000000,250.000000,111.803399,206.155281,250.000000
7,180.277564,250.000000,223.606798,50.000000,100.000000,70.710678,269.258240,0.000000,50.000000,269.258240,158.113883,111.803399,212.132034,150.000000,254.950976,70.710678,200.000000,158.113883,141.421356
8,158.113883,223.606798,206.155281,100.000000,50.000000,50.000000,254.950976,50.000000,0.000000,291.547595,150.000000,100.000000,180.277564,158.113883,269.258240,111.803399,206.155281,111.803399,180.277564
9,223.606798,254.950976,206.155281,254.950976,320.156212,250.000000,200.000000,269.258240,291.547595,0.000000,180.277564,212.132034,269.258240,141.421356,50.000000,206.155281,111.803399,320.156212,150.000000


# FIND CLOSEST TEAMS

In [161]:
#Find teams which are closest to each other
def find_closest_teams(DistanceTeamTeam, num_teams):
    closest_teams = {}
    for i in range(num_teams):
        for j in range(num_teams):
            if i != j:
                closest_teams[i, j] =  DistanceTeamTeam.iloc[i, j]
    closest_teams = dict(sorted(closest_teams.items(), key=lambda x: x[1]))
    return closest_teams

closest_teams = find_closest_teams(DistanceTeamTeam, n_teams)

In [162]:
closest_teams

{(2, 6): 49.999999999999986,
 (6, 2): 49.999999999999986,
 (7, 8): 49.999999999999986,
 (8, 7): 49.999999999999986,
 (9, 14): 49.999999999999986,
 (10, 11): 49.999999999999986,
 (10, 13): 49.999999999999986,
 (11, 10): 49.999999999999986,
 (13, 10): 49.999999999999986,
 (14, 9): 49.999999999999986,
 (0, 2): 50.0,
 (1, 12): 50.0,
 (2, 0): 50.0,
 (3, 7): 50.0,
 (7, 3): 50.0,
 (12, 1): 50.0,
 (13, 16): 50.0,
 (16, 13): 50.0,
 (0, 10): 50.00000000000001,
 (0, 12): 50.00000000000001,
 (1, 2): 50.00000000000001,
 (2, 1): 50.00000000000001,
 (3, 15): 50.00000000000001,
 (4, 8): 50.00000000000001,
 (5, 8): 50.00000000000001,
 (5, 11): 50.00000000000001,
 (8, 4): 50.00000000000001,
 (8, 5): 50.00000000000001,
 (10, 0): 50.00000000000001,
 (11, 5): 50.00000000000001,
 (12, 0): 50.00000000000001,
 (15, 3): 50.00000000000001,
 (10, 16): 70.71067811865473,
 (11, 13): 70.71067811865473,
 (13, 11): 70.71067811865473,
 (14, 16): 70.71067811865473,
 (16, 10): 70.71067811865473,
 (16, 14): 70.7106781186

In [163]:
type(closest_teams)

dict

In [180]:
all_teams = [i for i in range(0, n_teams)]


best_pairs = {}

for k in all_teams:
    for team_pairs in closest_teams.keys():
        if k in team_pairs:
            i, j = team_pairs
            print(i, j)
            best_pairs[team_pairs] = closest_teams[team_pairs]

            break
        

0 2
1 12
2 6
3 7
4 8
5 8
2 6
7 8
7 8
9 14
10 11
10 11
1 12
10 13
9 14
3 15
13 16
4 17
15 18


In [181]:
best_pairs

{(0, 2): 50.0,
 (1, 12): 50.0,
 (2, 6): 49.999999999999986,
 (3, 7): 50.0,
 (4, 8): 50.00000000000001,
 (5, 8): 50.00000000000001,
 (7, 8): 49.999999999999986,
 (9, 14): 49.999999999999986,
 (10, 11): 49.999999999999986,
 (10, 13): 49.999999999999986,
 (3, 15): 50.00000000000001,
 (13, 16): 50.0,
 (4, 17): 70.71067811865477,
 (15, 18): 70.71067811865474}

In [207]:
def best_swap(pair: tuple, expected_rewards: pd.DataFrame, current: pd.DataFrame): 
    i, j = pair
    i, j = i - 1, j - 1
    
    new_df = pd.DataFrame()
    
    new_df[i] = expected_rewards.iloc[i, :]
    new_df[j] = expected_rewards.iloc[j, :]
    
    
    current_i = current.iloc[i, :]
    current_j = current.iloc[j, :]
    
    new_df[i] = new_df[i] * (-1) * current_i
    new_df[j] = new_df[j] * (-1) * current_j    
        
        
    

In [187]:
rewards_df = pd.DataFrame(expected_rewards)
rewards_df

,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
0,136.074522,105.682739,93.000945,258.740714,103.621028,263.242359,302.823868,140.643303,72.444582,225.701586,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
2,136.074522,105.682739,93.000945,0.000000,0.000000,0.000000,302.823868,140.643303,72.444582,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000


In [201]:
rewards_df.iloc[1, :]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
939    0.0
940    0.0
941    0.0
942    0.0
943    0.0
Name: 1, Length: 944, dtype: float64

In [202]:
rewards_df.iloc[2, :]

0      136.074522
1      105.682739
2       93.000945
3        0.000000
4        0.000000
          ...    
939      0.000000
940      0.000000
941      0.000000
942      0.000000
943      0.000000
Name: 2, Length: 944, dtype: float64

In [228]:
new_df = pd.DataFrame()

In [229]:
new_df[1] = rewards_df.iloc[0, :]
new_df[3] = rewards_df.iloc[2, :]

In [230]:
new_df

,1,3
0,136.074522,136.074522
1,105.682739,105.682739
2,93.000945,93.000945
3,258.740714,0.000000
4,103.621028,0.000000
...,...,...
939,0.000000,0.000000
940,0.000000,0.000000
941,0.000000,0.000000
942,0.000000,0.000000


In [231]:
temp = SiteTeam.copy()
temp = temp.iloc[:, 1:]

In [232]:
temp

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [235]:
current = pd.DataFrame()

current[1] = temp.iloc[:, 0]
current[3] = temp.iloc[:, 2]

In [236]:
current

,1,3
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
939,0,0
940,0,0
941,0,0
942,0,0


In [239]:
feasible = new_df.copy()
feasible[feasible != 0] = 1
feasible

,1,3
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
939,0.0,0.0
940,0.0,0.0
941,0.0,0.0
942,0.0,0.0


In [258]:
for i in np.nonzero(current[1])[0]:
    if feasible.loc[i, 3] == 1:
    
        
        exchange_1_i = expected_rewards[0, i] * (-1)
        exchange_1_j = expected_rewards[2 ,i] * (-1)
        
        for j in np.nonzero(current[3])[0]:
            if feasible.loc[j, 1] == 1:
                exchange_2_i = expected_rewards[0, j] 
                exchange_2_j = expected_rewards[2, j]
                
                if exchange_1_i + exchange_2_j + exchange_1_j + exchange_2_i > 0:
                    print(f"Swap {i} and {j}")
                    break
        
                

Swap 2 and 71
Swap 8 and 51


In [259]:
def swap(cNum, expected_rewards, initial_solution, n_sites, n_teams = 2):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")
        capacity_1, capacity_2 = cNum
        
        x = {}
        a = expected_rewards
        
        for i in range(1, n_teams + 1):
            for j in range(1, n_sites):
                x[i, j] = m.addVar(vtype=GRB.BINARY, name="x_%s_%s" % (i, j))
        
        objective = gp.quicksum(a[i, j] * x[i, j] for i in range(1, n_teams + 1) for j in range(1, n_sites))
        
        
        m.addConstr(gp.quicksum(x[1, j] for j in range(1, n_sites + 1)) == capacity_1, name=f"c_{1}")
        m.addConstr(gp.quicksum(x[2, j] for j in range(1, n_sites + 1)) == capacity_2, name=f"c_{2}")
        
        
        for i in range(1, 3):
            m.addConstr(gp.quicksunm(x[i, j] * a[i, j] for j in range(1, n_sites + 1))>= gp.quicksum(initial_solution[i, j] * a[i, j] for j in range(1, n_sites + 1)), name=f"c__{i}")
            
        
        m.setObjective(objective, GRB.MAXIMIZE)
        
        
        m.optimize()
        
        
        if m.status == GRB.OPTIMAL:
            m.getAttr('x', x)   
            print('Obj: %g' % m.objVal)
            
        return m.objVal
        
        
        

SyntaxError: expected ':' (2571036341.py, line 8)